<a href="https://colab.research.google.com/github/GabrielDeml/trainingMobilenetV1SsdInColab/blob/master/wpilibCoralTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get classification transfer learning scripts.


In [14]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
!git clone https://github.com/tensorflow/models
%cd models/research/object_detection
!git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

Cloning into 'models'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 32607 (delta 25), reused 36 (delta 12), pack-reused 32547
Receiving objects: 100% (32607/32607), 511.87 MiB | 33.28 MiB/s, done.
Resolving deltas: 100% (20875/20875), done.
Checking out files: 100% (2486/2486), done.
/content/models/research/models/research/object_detection
Checking out files: 100% (1934/1934), done.
Note: checking out 'f788046ca876a8820e05b0b48c1fc2e16b0955bc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at f788046c Add model_dir to 

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [17]:
!CLASSIFY_DIR=/content/models/research/object_detection/
%pwd

'/content/models/research/models/research/object_detection'

In [18]:
!wget -O classify_scripts.tgz http://storage.googleapis.com/cloud-iot-edge-pretrained-models/docker/obj_det_scripts.tgz
!tar zxvf classify_scripts.tgz

--2020-03-09 17:29:32--  http://storage.googleapis.com/cloud-iot-edge-pretrained-models/docker/obj_det_scripts.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4622 (4.5K) [application/x-tar]
Saving to: ‘classify_scripts.tgz’

classify_scripts.tg 100%[===================>]   4.51K  --.-KB/s    in 0s      

2020-03-09 17:29:32 (457 MB/s) - ‘classify_scripts.tgz’ saved [4622/4622]

constants.sh
convert_checkpoint_to_edgetpu_tflite.sh
prepare_checkpoint_and_dataset.sh
retrain_detection_model.sh
workflow_test.sh
configs/
configs/pipeline_mobilenet_v2_ssd_retrain_whole_model.config
configs/pipeline_mobilenet_v1_ssd_retrain_last_few_layers.config
configs/pipeline_mobilenet_v2_ssd_retrain_last_few_layers.config
configs/pipeline_mobilenet_v1_ssd_retrain_whole_model.config


In [19]:
!sed -i '83s/object_detection\///' /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh
!sed -i '86s/object_detection\///' /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh

sed: can't read /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh: No such file or directory
sed: can't read /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh: No such file or directory


In [20]:
! ./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v1_ssd

+ network_type=mobilenet_v1_ssd
+ train_whole_model=false
+ [[ 2 -gt 0 ]]
+ case "$1" in
+ network_type=mobilenet_v1_ssd
+ shift 2
+ [[ 0 -gt 0 ]]
+ source /content/models/research/models/research/object_detection/constants.sh
++ declare -A ckpt_link_map
++ declare -A ckpt_name_map
++ declare -A config_filename_map
++ ckpt_link_map["mobilenet_v1_ssd"]=http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
++ ckpt_link_map["mobilenet_v2_ssd"]=http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
++ ckpt_name_map["mobilenet_v1_ssd"]=ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18
++ ckpt_name_map["mobilenet_v2_ssd"]=ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03
++ config_filename_map["mobilenet_v1_ssd-true"]=pipeline_mobilenet_v1_ssd_retrain_whole_model.config
++ config_filename_map["mobilenet_v1_ssd-false"]=pipeline_mobilenet_v1_ssd_retrain_last_few_

In [21]:
%tensorboard --logdir=./transfer_learn/train/

In [22]:
!./start_training.sh --network_type mobilenet_v1_ssd

/bin/bash: ./start_training.sh: No such file or directory


In [23]:
!./convert_checkpoint_to_edgetpu_tflite.sh --network_type mobilenet_v1_ssd --checkpoint_num 500

+ ckpt_number=0
+ [[ 4 -gt 0 ]]
+ case "$1" in
+ echo 'Unknown flag --network_type'
Unknown flag --network_type
+ usage
+ cat
  Converts TensorFlow checkpoint to EdgeTPU-compatible TFLite file.

  --checkpoint_num  Checkpoint number, by default 0.
  --help            Display this help.
+ exit 1


In [24]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!apt update
!apt install edgetpu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  14195      0 --:--:-- --:--:-- --:--:-- 14195
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Hit:1 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.

In [25]:
%cd /content/models/research/slim/transfer_learn/models/

[Errno 2] No such file or directory: '/content/models/research/slim/transfer_learn/models/'
/content/models/research/models/research/object_detection


In [26]:
!edgetpu_compiler output_tflite_graph.tflite

Error opening file for reading: output_tflite_graph.tflite


In [27]:
%cd python-tflite-source/edgetpu/demo/
!python3 classify_image.py \
  --model ${CLASSIFY_DIR}/models/output_tflite_graph_edgetpu.tflite \
  --label ${CLASSIFY_DIR}/models/labels.txt \
  --input ${CLASSIFY_DIR}/flower.jpg

[Errno 2] No such file or directory: 'python-tflite-source/edgetpu/demo/'
/content/models/research/models/research/object_detection
python3: can't open file 'classify_image.py': [Errno 2] No such file or directory
